# NB02: All Constraints Training v2.0

**Constraint-Based Architectural NCA - Step B v2.0**

**Version:** 2.0 (Ontology Revision)  
**Date:** December 2025  
**Purpose:** Fix degenerate attractor via loss ontology restructuring

---

## Why v2.0? Ontology Problem Identified

v1.0-v1.1 produced **stable degenerate attractors** because:
1. **Legality was global** - Scalar ratios can be gamed
2. **Growth was misaligned** - Incentives conflicted with rules
3. **Connectivity was boundary-seeded** - Perimeter corridor exploit
4. **Structure was mass-based** - No causal load paths

## v2.0 Core Principle

**Every constraint must be LOCAL and CAUSAL.**

## New Loss Architecture

| Category | Loss | Replaces | Weight |
|----------|------|----------|--------|
| **Legality** | LocalLegality | StreetVoid + Anchor | 30.0 |
| **Growth** | AlignedGrowth | Dice | 25.0 |
| **Structure** | LoadPath | Support + Connectivity | 20.0 |
| **Circulation** | AccessConnectivity | StreetConnectivity | 15.0 |
| **Massing** | Sparsity | - | 15.0 |
| **Structure** | Cantilever | - | 5.0 |
| **Quality** | Density | - | 3.0 |
| **Quality** | TV | - | 1.0 |

## Success Criteria

| Metric | Target |
|--------|--------|
| Legality | 100% (all structure in legal zones) |
| Growth alignment | >80% in legal target |
| Load path | >95% connected to support |
| Access connectivity | >90% mutual reachability |
| Fill ratio | 5-15% |

---

## 1. Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = '/content/drive/MyDrive/Constraint-NCA'
print(f'Project root: {PROJECT_ROOT}')

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from typing import Dict, Tuple, Optional, List
import json
from datetime import datetime
from tqdm.notebook import tqdm
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# Set seeds
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [ ]:
# Load Step B configuration
with open(f'{PROJECT_ROOT}/config_step_b.json', 'r') as f:
    CONFIG = json.load(f)

# Training configuration
CONFIG.update({
    'epochs': 400,
    'steps_min': 30,
    'steps_max': 50,
    'difficulty': 'easy',
    'log_every': 20,
    'viz_every': 100,
    'save_every': 100,
})

print('Configuration loaded')
print(f"Difficulty: {CONFIG['difficulty']}")
print(f"Epochs: {CONFIG['epochs']}")

## 2. Foundation Components (Unchanged from v1.1)

In [ ]:
# ============================================================
# PERCEPTION MODULE
# ============================================================

class Perceive3D(nn.Module):
    """3D Sobel perception with replicate padding."""

    def __init__(self, n_channels: int = 8):
        super().__init__()
        self.n_channels = n_channels

        sobel_x = self._create_sobel_kernel('x')
        sobel_y = self._create_sobel_kernel('y')
        sobel_z = self._create_sobel_kernel('z')
        identity = self._create_identity_kernel()

        kernels = torch.stack([identity, sobel_x, sobel_y, sobel_z], dim=0)
        self.register_buffer('kernels', kernels)

    def _create_sobel_kernel(self, direction: str) -> torch.Tensor:
        derivative = torch.tensor([-1., 0., 1.])
        smoothing = torch.tensor([1., 2., 1.])

        if direction == 'x':
            kernel = torch.einsum('i,j,k->ijk', smoothing, smoothing, derivative)
        elif direction == 'y':
            kernel = torch.einsum('i,j,k->ijk', smoothing, derivative, smoothing)
        elif direction == 'z':
            kernel = torch.einsum('i,j,k->ijk', derivative, smoothing, smoothing)
        return kernel / 16.0

    def _create_identity_kernel(self) -> torch.Tensor:
        kernel = torch.zeros(3, 3, 3)
        kernel[1, 1, 1] = 1.0
        return kernel

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, D, H, W = x.shape
        x_padded = F.pad(x, (1, 1, 1, 1, 1, 1), mode='replicate')
        
        outputs = []
        for k in range(4):
            kernel = self.kernels[k:k+1].unsqueeze(0).expand(C, 1, 3, 3, 3)
            out = F.conv3d(x_padded, kernel, padding=0, groups=C)
            outputs.append(out)
        return torch.cat(outputs, dim=1)

In [ ]:
# ============================================================
# NCA MODEL
# ============================================================

class UrbanPavilionNCA(nn.Module):
    """Neural Cellular Automaton for urban pavilion generation."""

    def __init__(self, config: dict):
        super().__init__()
        self.config = config

        n_channels = config['n_channels']
        hidden_dim = config['hidden_dim']
        perception_dim = n_channels * 4
        n_grown = config['n_grown']

        self.perceive = Perceive3D(n_channels)

        self.update_net = nn.Sequential(
            nn.Conv3d(perception_dim, hidden_dim, 1),
            nn.ReLU(),
            nn.Conv3d(hidden_dim, hidden_dim, 1),
            nn.ReLU(),
            nn.Conv3d(hidden_dim, n_grown, 1),
        )

        self._init_weights()

    def _init_weights(self):
        gain = self.config['xavier_gain']
        for m in self.update_net:
            if isinstance(m, nn.Conv3d):
                nn.init.xavier_uniform_(m.weight, gain=gain)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

        last_layer = self.update_net[-1]
        with torch.no_grad():
            last_layer.bias[0] = self.config['structure_bias']
            last_layer.bias[1] = self.config['surface_bias']

    def forward(self, state: torch.Tensor, steps: int = 1) -> torch.Tensor:
        for _ in range(steps):
            state = self._step(state)
        return state

    def _step(self, state: torch.Tensor) -> torch.Tensor:
        B, C, D, H, W = state.shape
        cfg = self.config

        perception = self.perceive(state)
        delta = self.update_net(perception)

        if self.training:
            fire_mask = (torch.rand(B, 1, D, H, W, device=state.device) < cfg['fire_rate']).float()
            delta = delta * fire_mask

        grown_start = cfg['n_frozen']
        grown_new = state[:, grown_start:] + cfg['update_scale'] * delta
        grown_new = torch.clamp(grown_new, 0.0, 1.0)

        # Hard mask: no structure inside buildings
        existing = state[:, cfg['ch_existing']:cfg['ch_existing']+1]
        available_mask = 1.0 - existing
        struct_new = grown_new[:, 0:1] * available_mask
        
        grown_masked = torch.cat([struct_new, grown_new[:, 1:]], dim=1)
        new_state = torch.cat([state[:, :grown_start], grown_masked], dim=1)

        return new_state

    def grow(self, seed: torch.Tensor, steps: int = 50) -> torch.Tensor:
        self.eval()
        with torch.no_grad():
            return self.forward(seed, steps)

In [ ]:
# ============================================================
# SCENE GENERATOR
# ============================================================

class UrbanSceneGenerator:
    """Generate urban scenes with buildings, access points, and anchor zones."""

    def __init__(self, config: dict):
        self.config = config
        self.G = config['grid_size']
        self.C = config['n_channels']

    def generate(self, difficulty: str = 'easy',
                 device: str = 'cuda') -> Tuple[torch.Tensor, dict]:
        G = self.G
        cfg = self.config
        
        state = torch.zeros(1, self.C, G, G, G, device=device)
        state[:, cfg['ch_ground'], 0, :, :] = 1.0

        params = self._get_difficulty_params(difficulty)
        building_info = self._place_buildings(state, params)
        access_info = self._place_access_points(state, params, building_info)
        anchor_info = self._generate_anchor_zones(state, params, building_info, access_info)

        metadata = {
            'difficulty': difficulty,
            'buildings': building_info,
            'access_points': access_info,
            'anchor_zones': anchor_info,
            'gap_width': params['gap_width'],
        }
        
        return state, metadata

    def _get_difficulty_params(self, difficulty: str) -> dict:
        G = self.G
        if difficulty == 'easy':
            return {
                'n_buildings': 2, 'height_range': (12, 16), 'height_variance': False,
                'width_range': (8, 12), 'gap_width': random.randint(14, 18),
                'n_ground_access': 1, 'n_elevated_access': 1, 'anchor_budget': 0.10,
            }
        elif difficulty == 'medium':
            return {
                'n_buildings': 2, 'height_range': (10, 20), 'height_variance': True,
                'width_range': (6, 10), 'gap_width': random.randint(10, 14),
                'n_ground_access': random.randint(1, 2), 'n_elevated_access': random.randint(1, 2),
                'anchor_budget': 0.07,
            }
        else:  # hard
            return {
                'n_buildings': random.randint(2, 4), 'height_range': (8, 24), 'height_variance': True,
                'width_range': (5, 8), 'gap_width': random.randint(6, 10),
                'n_ground_access': random.randint(2, 3), 'n_elevated_access': random.randint(2, 3),
                'anchor_budget': 0.05,
            }

    def _place_buildings(self, state: torch.Tensor, params: dict) -> list:
        G = self.G
        ch = self.config['ch_existing']
        buildings = []
        gap_width = params['gap_width']
        gap_center = G // 2

        w1 = random.randint(*params['width_range'])
        d1 = random.randint(G//2, G-2)
        h1 = random.randint(*params['height_range'])
        x1_end = gap_center - gap_width // 2
        x1_start = max(0, x1_end - w1)
        state[:, ch, :h1, :d1, x1_start:x1_end] = 1.0
        buildings.append({'x': (x1_start, x1_end), 'y': (0, d1), 'z': (0, h1),
                         'gap_facing_x': x1_end, 'side': 'left'})

        w2 = random.randint(*params['width_range'])
        d2 = random.randint(G//2, G-2)
        h2 = h1 if not params['height_variance'] else random.randint(*params['height_range'])
        x2_start = gap_center + gap_width // 2
        x2_end = min(G, x2_start + w2)
        state[:, ch, :h2, :d2, x2_start:x2_end] = 1.0
        buildings.append({'x': (x2_start, x2_end), 'y': (0, d2), 'z': (0, h2),
                         'gap_facing_x': x2_start, 'side': 'right'})

        return buildings

    def _place_access_points(self, state: torch.Tensor, params: dict, buildings: list) -> list:
        G = self.G
        ch = self.config['ch_access']
        access_points = []
        
        left_buildings = [b for b in buildings if b['side'] == 'left']
        right_buildings = [b for b in buildings if b['side'] == 'right']
        gap_x_min = max(b['gap_facing_x'] for b in left_buildings) if left_buildings else 0
        gap_x_max = min(b['gap_facing_x'] for b in right_buildings) if right_buildings else G
        
        for i in range(params.get('n_ground_access', 1)):
            x = random.randint(gap_x_min + 1, gap_x_max - 3)
            y = random.randint(0, G - 3)
            state[:, ch, 0:2, y:y+2, x:x+2] = 1.0
            access_points.append({'x': x, 'y': y, 'z': 0, 'type': 'ground'})
        
        for i in range(params.get('n_elevated_access', 1)):
            building = random.choice(buildings)
            bz_max = building['z'][1]
            is_left = building['side'] == 'left'
            
            z = random.randint(3, max(4, bz_max - 2))
            y = random.randint(building['y'][0], min(building['y'][1] - 2, building['y'][0] + G//3))
            x = building['x'][1] if is_left else building['x'][0] - 2
            x = max(0, min(G - 2, x))
            
            state[:, ch, z:z+2, y:y+2, x:x+2] = 1.0
            access_points.append({'x': x, 'y': y, 'z': z, 'type': 'elevated'})

        return access_points

    def _generate_anchor_zones(self, state: torch.Tensor, params: dict,
                               buildings: list, access_points: list) -> dict:
        G = self.G
        ch = self.config['ch_anchors']
        street_levels = self.config['street_levels']
        
        existing = state[:, self.config['ch_existing'], 0, :, :]
        street_mask = 1.0 - existing
        total_street_area = street_mask.sum().item()
        max_anchor_area = int(total_street_area * params['anchor_budget'])
        
        anchors = torch.zeros(1, 1, G, G, G, device=state.device)
        current_anchor_area = 0
        
        for ap in access_points:
            if ap['type'] == 'ground':
                x, y = ap['x'], ap['y']
                for z in range(street_levels):
                    anchors[:, 0, z, max(0,y-1):min(G,y+3), max(0,x-1):min(G,x+3)] = 1.0
        
        for building in buildings:
            by_start, by_end = building['y']
            gap_x = building['gap_facing_x']
            is_left = building['side'] == 'left'
            x_start = gap_x if is_left else gap_x - 1
            x_end = gap_x + 1 if is_left else gap_x
            for z in range(street_levels):
                anchors[:, 0, z, by_start:by_end, max(0,x_start):min(G,x_end)] = 1.0
        
        for z in range(street_levels):
            anchors[:, 0, z, :, :] *= street_mask
        
        state[:, ch:ch+1, :, :, :] = anchors
        final_anchor_area = (anchors > 0.5).sum().item()
        
        return {'total_area': final_anchor_area, 'budget': max_anchor_area,
                'budget_ratio': params['anchor_budget'], 'street_area': total_street_area}

    def batch(self, difficulty: str, batch_size: int, device: str) -> torch.Tensor:
        scenes = [self.generate(difficulty, device)[0] for _ in range(batch_size)]
        return torch.cat(scenes, dim=0)


print('Foundation components loaded')

## 3. v2.0 Loss Functions (Ontology Revision)

### Key Changes from v1.x

| Old Loss | Problem | New Loss | Fix |
|----------|---------|----------|-----|
| StreetVoidLoss | Global ratio, gameable | LocalLegalityLoss | Per-voxel field |
| AnchorBudgetLoss | Normalized, gameable | LocalLegalityLoss | Per-voxel field |
| DiceLoss | Not aligned with legality | AlignedGrowthLoss | Legal ∩ Access |
| StreetConnectivityLoss | Boundary-seeded | AccessConnectivityLoss | Access-seeded |
| SupportRatioLoss | Mass-based | LoadPathLoss | Causal path tracing |
| ConnectivityLoss | Includes ground | LoadPathLoss | Buildings + anchors only |

In [ ]:
# ============================================================
# CONSTRAINT 1: LOCAL LEGALITY LOSS (NEW - REPLACES VOID + ANCHOR)
# ============================================================

class LocalLegalityLoss(nn.Module):
    """Per-voxel legality enforcement.
    
    REPLACES: StreetVoidLoss, AnchorBudgetLoss
    
    Every voxel has a binary legality value:
    - Legal (1): Above street level, OR in anchor zone at street level
    - Illegal (0): At street level AND outside anchors, OR inside buildings
    
    Key insight: No global ratios to game. Each illegal voxel is penalized directly.
    """

    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.street_levels = config['street_levels']

    def compute_legality_field(self, state: torch.Tensor) -> torch.Tensor:
        """Compute per-voxel legality (1 = legal, 0 = illegal).
        
        Rules:
        - Inside buildings: ILLEGAL
        - At street level (z < 2) outside anchors: ILLEGAL
        - At street level (z < 2) inside anchors: LEGAL
        - Above street level (z >= 2) outside buildings: LEGAL
        """
        cfg = self.config
        G = cfg['grid_size']
        street_levels = self.street_levels

        existing = state[:, cfg['ch_existing']]
        anchors = state[:, cfg['ch_anchors']]

        # Create height mask (1 for above street, 0 for street level)
        # Using broadcasting instead of in-place operations
        B = state.shape[0]
        device = state.device
        
        # Create z-coordinate tensor
        z_indices = torch.arange(G, device=device).view(1, G, 1, 1).expand(B, G, G, G)
        
        # Above street level mask (z >= street_levels)
        above_street = (z_indices >= street_levels).float()
        
        # At street level mask (z < street_levels)
        at_street = (z_indices < street_levels).float()
        
        # Position legality:
        # - Above street: always legal (1)
        # - At street: legal only in anchors
        position_legality = above_street + at_street * anchors
        
        # Final legality = not in buildings AND position is legal
        legality = (1 - existing) * position_legality
        legality = torch.clamp(legality, 0, 1)

        return legality

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        structure = state[:, cfg['ch_structure']]

        legality = self.compute_legality_field(state)

        # Direct per-voxel penalty: structure in illegal zones
        illegal_structure = structure * (1 - legality)

        # Normalize by total structure (but NOT in a way that can be gamed)
        # Key: numerator increases with violations, can't be reduced by adding more structure
        return illegal_structure.sum() / (structure.sum() + 1e-8)


print('LocalLegalityLoss defined (replaces StreetVoid + AnchorBudget)')

In [ ]:
# ============================================================
# CONSTRAINT 2: ALIGNED GROWTH LOSS (NEW - REPLACES DICE)
# ============================================================

class AlignedGrowthLoss(nn.Module):
    """Growth incentive aligned with legality.
    
    REPLACES: DiceLoss, AccessReachLoss, ElevatedBonus
    
    Growth is rewarded ONLY in zones that are BOTH:
    1. Near access points (functionally useful)
    2. Legal (permitted by constraints)
    
    Key insight: Growth incentive and legality point the same direction.
    The model cannot be rewarded for illegal growth.
    """

    def __init__(self, config: dict, dilation_radius: int = 5):
        super().__init__()
        self.config = config
        self.dilation_radius = dilation_radius

    def forward(self, state: torch.Tensor, legality_field: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        access = state[:, cfg['ch_access']]
        existing = state[:, cfg['ch_existing']]
        G = cfg['grid_size']

        # Dilate access points to create influence zone
        k = 2 * self.dilation_radius + 1
        access_zone = F.max_pool3d(access.unsqueeze(1), k, 1, self.dilation_radius).squeeze(1)

        # Legal growth target = access zone AND legal AND not in buildings
        available = 1.0 - existing
        legal_target = access_zone * legality_field * available

        # Optional: height weighting to encourage elevated structures
        z_coords = torch.arange(G, device=state.device).float()
        height_weight = (z_coords / G).view(1, G, 1, 1).expand(1, G, G, G)
        legal_target_weighted = legal_target * (1 + 0.5 * height_weight)

        # Dice loss toward legal target only
        intersection = (structure * legal_target_weighted).sum()
        dice = (2 * intersection + 1) / (structure.sum() + legal_target_weighted.sum() + 1)

        return 1 - dice


print('AlignedGrowthLoss defined (replaces Dice + AccessReach + ElevatedBonus)')

In [ ]:
# ============================================================
# CONSTRAINT 3: ACCESS CONNECTIVITY LOSS (NEW - REPLACES STREET CONN)
# ============================================================

class AccessConnectivityLoss(nn.Module):
    """Street connectivity seeded from access points.
    
    REPLACES: StreetConnectivityLoss
    
    Measures whether all access points are mutually reachable
    through the void network, NOT just boundary-to-boundary.
    
    Key insight: A thin perimeter corridor no longer helps.
    Connectivity is functional (access-to-access), not topological.
    """

    def __init__(self, config: dict, iterations: int = 32):
        super().__init__()
        self.config = config
        self.street_levels = config['street_levels']
        self.iterations = iterations

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        street_levels = self.street_levels

        # Get relevant channels at street level
        structure = state[:, cfg['ch_structure'], :street_levels, :, :]
        existing = state[:, cfg['ch_existing'], :street_levels, :, :]
        access = state[:, cfg['ch_access'], :street_levels, :, :]

        # Void mask = not structure AND not existing buildings
        void_mask = (1 - structure) * (1 - existing)

        # Seed from ACCESS POINTS (not boundaries!)
        # Dilate access slightly to ensure seed overlaps with void
        access_seed = F.max_pool3d(access.unsqueeze(1), 3, 1, 1).squeeze(1)
        connected = access_seed * void_mask

        # Flood fill through void
        for _ in range(self.iterations):
            dilated = F.max_pool3d(connected.unsqueeze(1), 3, 1, 1).squeeze(1)
            new_connected = torch.max(connected, dilated * void_mask)
            if torch.allclose(connected, new_connected, atol=1e-5):
                break
            connected = new_connected

        # Check: can we reach ALL access points from each other?
        access_locations = (access > 0.5).float()
        reachable = (connected * access_locations).sum()
        total_access = access_locations.sum() + 1e-8

        # Loss = fraction of access points NOT reachable from the flood
        return 1 - (reachable / total_access)


print('AccessConnectivityLoss defined (replaces StreetConnectivityLoss)')

In [ ]:
# ============================================================
# CONSTRAINT 4: LOAD PATH LOSS (NEW - REPLACES SUPPORT + CONNECTIVITY)
# ============================================================

class LoadPathLoss(nn.Module):
    """Structural load path connectivity.
    
    REPLACES: SupportRatioLoss, ConnectivityLoss
    
    Elevated mass must be connected to ground support (anchors + buildings)
    through continuous structure - not just co-existing.
    
    Key insight: Mass statistics can be gamed. Load paths cannot.
    This traces actual structural paths, not just counts voxels.
    """

    def __init__(self, config: dict, iterations: int = 32):
        super().__init__()
        self.config = config
        self.street_levels = config['street_levels']
        self.iterations = iterations

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        street_levels = self.street_levels
        G = cfg['grid_size']

        structure = state[:, cfg['ch_structure']]
        existing = state[:, cfg['ch_existing']]
        anchors = state[:, cfg['ch_anchors']]

        # Valid support = buildings + anchors (at street level only)
        # NOT free ground! This is the key fix.
        # Use cat instead of in-place assignment to avoid gradient issues
        B = state.shape[0]
        device = state.device
        
        # At street level: support = max(existing, anchors)
        support_street = torch.max(
            existing[:, :street_levels, :, :],
            anchors[:, :street_levels, :, :]
        )
        
        # Above street level: support = existing buildings only
        support_above = existing[:, street_levels:, :, :]
        
        # Concatenate to form full support tensor (no in-place ops)
        support = torch.cat([support_street, support_above], dim=1)

        # Flood fill through STRUCTURE from support
        # This traces load paths, not just checks adjacency
        connected = support.clone()
        
        # Use soft threshold for gradient flow
        struct_soft = torch.sigmoid(10 * (structure - 0.3))

        for _ in range(self.iterations):
            dilated = F.max_pool3d(connected.unsqueeze(1), 3, 1, 1).squeeze(1)
            # Can only spread through structure
            new_connected = torch.max(connected, dilated * struct_soft)
            if torch.allclose(connected, new_connected, atol=1e-5):
                break
            connected = new_connected

        # Elevated structure = structure at z >= street_levels
        elevated = structure[:, street_levels:, :, :]
        elevated_connected = connected[:, street_levels:, :, :]

        # Loss = elevated structure NOT connected to support via load path
        unsupported = elevated * (1 - elevated_connected)

        return unsupported.sum() / (elevated.sum() + 1e-8)


print('LoadPathLoss defined (replaces SupportRatio + Connectivity)')

In [ ]:
# ============================================================
# CONSTRAINT 5: CANTILEVER LOSS (UNCHANGED)
# ============================================================

class CantileverLoss(nn.Module):
    """Limit horizontal overhangs.
    
    Each voxel must have support within N voxels below.
    Works together with LoadPath:
    - Cantilever: local support requirement
    - LoadPath: global path to ground requirement
    """

    def __init__(self, max_overhang: int = 3, threshold: float = 0.3):
        super().__init__()
        self.max_overhang = max_overhang
        self.threshold = threshold

    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        B, D, H, W = structure.shape
        N = self.max_overhang
        total_loss = 0.0
        count = 0

        for z in range(N, D):
            layer = structure[:, z]
            support_volume = structure[:, max(0, z-N):z]
            support_max = support_volume.max(dim=1)[0]
            support_dilated = F.max_pool2d(support_max.unsqueeze(1), 3, 1, 1).squeeze(1)
            has_support = torch.sigmoid(10 * (support_dilated - self.threshold))
            unsupported = layer * (1.0 - has_support)
            total_loss += unsupported.mean()
            count += 1

        return total_loss / max(1, count)


print('CantileverLoss defined (unchanged)')

In [ ]:
# ============================================================
# CONSTRAINT 6: SPARSITY LOSS (UNCHANGED)
# ============================================================

class SparsityLoss(nn.Module):
    """Limit total volume (5-15% fill ratio)."""

    def __init__(self, max_ratio: float = 0.15, min_ratio: float = 0.05, 
                 under_coef: float = 3.0):
        super().__init__()
        self.max_ratio = max_ratio
        self.min_ratio = min_ratio
        self.under_coef = under_coef

    def forward(self, structure: torch.Tensor, available: torch.Tensor) -> torch.Tensor:
        ratio = structure.sum() / (available.sum() + 1e-8)
        over_penalty = F.relu(ratio - self.max_ratio)
        under_penalty = self.under_coef * F.relu(self.min_ratio - ratio)
        return over_penalty + under_penalty


print('SparsityLoss defined (unchanged)')

In [ ]:
# ============================================================
# QUALITY LOSSES (REDUCED WEIGHTS)
# ============================================================

class DensityPenalty(nn.Module):
    """SIMP density penalty for binary outputs."""
    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        return (structure * (1.0 - structure)).mean()


class TotalVariation3D(nn.Module):
    """Total variation for smooth surfaces."""
    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        tv_d = (structure[:, 1:, :, :] - structure[:, :-1, :, :]).abs().mean()
        tv_h = (structure[:, :, 1:, :] - structure[:, :, :-1, :]).abs().mean()
        tv_w = (structure[:, :, :, 1:] - structure[:, :, :, :-1]).abs().mean()
        return tv_d + tv_h + tv_w


print('Quality losses defined (with reduced weights)')

In [ ]:
# Test all v2.0 losses
print('Testing v2.0 loss functions...')
print('='*60)

scene_gen = UrbanSceneGenerator(CONFIG)
scene, meta = scene_gen.generate('easy', device)

# Add test structure
test_state = scene.clone()
# Legal elevated structure
test_state[:, CONFIG['ch_structure'], 2:10, 10:20, 12:20] = 0.6
# Some ground contact in anchors
anchors = test_state[:, CONFIG['ch_anchors']]
test_state[:, CONFIG['ch_structure'], 0:2, :, :] = 0.6 * anchors[:, 0:2, :, :]
# Add some illegal ground structure for testing
test_state[:, CONFIG['ch_structure'], 0, 15:18, 15:18] = 0.5

structure = test_state[:, CONFIG['ch_structure']]
existing = test_state[:, CONFIG['ch_existing']]
available = 1.0 - existing

# Test LocalLegalityLoss
legality_loss = LocalLegalityLoss(CONFIG)
legality_field = legality_loss.compute_legality_field(test_state)
L_legality = legality_loss(test_state)
print(f'LocalLegality: {L_legality.item():.4f}')
print(f'  Legality field coverage: {legality_field.mean().item():.2%}')

# Test AlignedGrowthLoss
growth_loss = AlignedGrowthLoss(CONFIG)
L_growth = growth_loss(test_state, legality_field)
print(f'AlignedGrowth: {L_growth.item():.4f}')

# Test AccessConnectivityLoss
access_conn_loss = AccessConnectivityLoss(CONFIG)
L_access_conn = access_conn_loss(test_state)
print(f'AccessConnectivity: {L_access_conn.item():.4f}')

# Test LoadPathLoss
loadpath_loss = LoadPathLoss(CONFIG)
L_loadpath = loadpath_loss(test_state)
print(f'LoadPath: {L_loadpath.item():.4f}')

# Test unchanged losses
L_cant = CantileverLoss()(structure)
print(f'Cantilever: {L_cant.item():.4f}')

L_sparse = SparsityLoss()(structure, available)
print(f'Sparsity: {L_sparse.item():.4f}')

L_density = DensityPenalty()(structure)
print(f'Density: {L_density.item():.4f}')

L_tv = TotalVariation3D()(structure)
print(f'TV: {L_tv.item():.4f}')

print('='*60)
print('All 8 v2.0 loss functions working')

## 4. Evaluation Metrics (v2.0)

In [ ]:
def compute_legality_compliance(state: torch.Tensor, config: dict) -> torch.Tensor:
    """Fraction of structure that is in legal zones.
    Target: 100%
    """
    legality_loss = LocalLegalityLoss(config)
    legality_field = legality_loss.compute_legality_field(state)
    structure = state[:, config['ch_structure']]
    
    legal_structure = (structure * legality_field).sum()
    total_structure = structure.sum() + 1e-8
    
    return legal_structure / total_structure


def compute_growth_alignment(state: torch.Tensor, config: dict) -> torch.Tensor:
    """Fraction of structure in legal target zones.
    Target: >80%
    """
    legality_loss = LocalLegalityLoss(config)
    legality_field = legality_loss.compute_legality_field(state)
    
    structure = state[:, config['ch_structure']]
    access = state[:, config['ch_access']]
    existing = state[:, config['ch_existing']]
    
    # Access zone
    access_zone = F.max_pool3d(access.unsqueeze(1), 11, 1, 5).squeeze(1)
    available = 1.0 - existing
    
    # Legal target
    legal_target = access_zone * legality_field * available
    
    # Structure in legal target
    aligned = (structure * legal_target).sum()
    total = structure.sum() + 1e-8
    
    return aligned / total


def compute_load_path_compliance(state: torch.Tensor, config: dict) -> torch.Tensor:
    """Fraction of elevated structure connected via load path.
    Target: >95%
    """
    loadpath = LoadPathLoss(config)
    loss = loadpath(state)
    return 1.0 - loss


def compute_access_reachability(state: torch.Tensor, config: dict) -> torch.Tensor:
    """Fraction of access points mutually reachable.
    Target: >90%
    """
    access_conn = AccessConnectivityLoss(config)
    loss = access_conn(state)
    return 1.0 - loss


def compute_fill_ratio(state: torch.Tensor, config: dict) -> torch.Tensor:
    """Total volume ratio.
    Target: 5-15%
    """
    structure = state[:, config['ch_structure']]
    existing = state[:, config['ch_existing']]
    available = 1.0 - existing
    return structure.sum() / (available.sum() + 1e-8)


print('v2.0 evaluation metrics defined')

## 5. Trainer (v2.0 Ontology)

In [ ]:
class OntologyTrainer:
    """Step B v2.0 Trainer: Ontology-revised loss functions.
    
    Key changes from v1.x:
    - LocalLegalityLoss replaces StreetVoid + Anchor (per-voxel, not global)
    - AlignedGrowthLoss replaces Dice (growth aligned with legality)
    - AccessConnectivityLoss replaces StreetConn (access-seeded, not boundary)
    - LoadPathLoss replaces Support + Connectivity (causal paths, not mass)
    """

    def __init__(self, model: nn.Module, config: dict, device: str):
        self.model = model
        self.config = config
        self.device = device

        # v2.0 loss functions
        self.legality_loss = LocalLegalityLoss(config)
        self.growth_loss = AlignedGrowthLoss(config, dilation_radius=5)
        self.access_conn_loss = AccessConnectivityLoss(config)
        self.loadpath_loss = LoadPathLoss(config)
        self.cantilever_loss = CantileverLoss()
        self.sparsity_loss = SparsityLoss(max_ratio=0.15, min_ratio=0.05, under_coef=3.0)
        self.density_loss = DensityPenalty()
        self.tv_loss = TotalVariation3D()

        # v2.0 weights (from CONSTRAINT_SPECS v3.0)
        self.weights = {
            'legality': 30.0,       # Per-voxel legality (replaces void + anchor)
            'growth': 25.0,         # Aligned growth (replaces dice)
            'loadpath': 20.0,       # Load path (replaces support + connectivity)
            'access_conn': 15.0,    # Access connectivity (replaces street conn)
            'sparsity': 15.0,       # Volume bounds
            'cantilever': 5.0,      # Local overhang
            'density': 3.0,         # Binary (reduced from 5)
            'tv': 1.0,              # Smoothness
        }

        self.optimizer = torch.optim.Adam(model.parameters(), lr=config['lr_initial'])
        self.scene_gen = UrbanSceneGenerator(config)
        self.history = []

    def train_epoch(self, epoch: int) -> dict:
        self.model.train()
        cfg = self.config

        # Generate batch
        seeds = self.scene_gen.batch(cfg['difficulty'], cfg['batch_size'], self.device)
        steps = random.randint(cfg['steps_min'], cfg['steps_max'])

        # Forward pass
        final = self.model(seeds, steps=steps)

        # Extract channels
        structure = final[:, cfg['ch_structure']]
        existing = final[:, cfg['ch_existing']]
        available = 1.0 - existing

        # Compute legality field (used by multiple losses)
        legality_field = self.legality_loss.compute_legality_field(final)

        # Compute v2.0 losses
        L_legality = self.legality_loss(final)
        L_growth = self.growth_loss(final, legality_field)
        L_access_conn = self.access_conn_loss(final)
        L_loadpath = self.loadpath_loss(final)
        L_cant = self.cantilever_loss(structure)
        L_sparse = self.sparsity_loss(structure, available)
        L_density = self.density_loss(structure)
        L_tv = self.tv_loss(structure)

        # Total loss with v2.0 weights
        total_loss = (
            self.weights['legality'] * L_legality +
            self.weights['growth'] * L_growth +
            self.weights['loadpath'] * L_loadpath +
            self.weights['access_conn'] * L_access_conn +
            self.weights['sparsity'] * L_sparse +
            self.weights['cantilever'] * L_cant +
            self.weights['density'] * L_density +
            self.weights['tv'] * L_tv
        )

        # Backward pass
        self.optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), cfg['grad_clip'])
        self.optimizer.step()

        # Compute metrics
        with torch.no_grad():
            legality_compliance = compute_legality_compliance(final, cfg).item()
            growth_alignment = compute_growth_alignment(final, cfg).item()
            loadpath_compliance = compute_load_path_compliance(final, cfg).item()
            access_reachability = compute_access_reachability(final, cfg).item()
            fill_ratio = compute_fill_ratio(final, cfg).item()

        metrics = {
            'epoch': epoch,
            'total_loss': total_loss.item(),
            # Losses
            'L_legality': L_legality.item(),
            'L_growth': L_growth.item(),
            'L_loadpath': L_loadpath.item(),
            'L_access_conn': L_access_conn.item(),
            'L_sparsity': L_sparse.item(),
            'L_cantilever': L_cant.item(),
            'L_density': L_density.item(),
            'L_tv': L_tv.item(),
            # Metrics
            'legality_compliance': legality_compliance,
            'growth_alignment': growth_alignment,
            'loadpath_compliance': loadpath_compliance,
            'access_reachability': access_reachability,
            'fill_ratio': fill_ratio,
            'steps': steps,
        }

        self.history.append(metrics)
        return metrics

    def evaluate(self, n_samples: int = 20) -> dict:
        self.model.eval()
        cfg = self.config
        results = []

        with torch.no_grad():
            for _ in range(n_samples):
                scene, meta = self.scene_gen.generate(cfg['difficulty'], self.device)
                grown = self.model.grow(scene, steps=50)
                
                results.append({
                    'legality': compute_legality_compliance(grown, cfg).item(),
                    'growth_align': compute_growth_alignment(grown, cfg).item(),
                    'loadpath': compute_load_path_compliance(grown, cfg).item(),
                    'access_reach': compute_access_reachability(grown, cfg).item(),
                    'fill_ratio': compute_fill_ratio(grown, cfg).item(),
                })

        return {
            'avg_legality': np.mean([r['legality'] for r in results]),
            'avg_growth_alignment': np.mean([r['growth_align'] for r in results]),
            'avg_loadpath': np.mean([r['loadpath'] for r in results]),
            'avg_access_reach': np.mean([r['access_reach'] for r in results]),
            'avg_fill_ratio': np.mean([r['fill_ratio'] for r in results]),
            'n_samples': n_samples,
        }

    def save_checkpoint(self, path: str):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'config': self.config,
            'history': self.history,
            'weights': self.weights,
        }, path)
        print(f'Checkpoint saved: {path}')


print('OntologyTrainer defined')

## 6. Visualization (Updated for v2.0 Metrics)

In [ ]:
def visualize_v2_result(model, scene_gen, config, device, title='Result'):
    """Visualize grown structure with v2.0 metrics."""
    model.eval()
    scene, meta = scene_gen.generate(config['difficulty'], device)
    
    with torch.no_grad():
        grown = model.grow(scene, steps=50)
    
    cfg = config
    s = grown[0].cpu().numpy()
    G = s.shape[1]
    street_levels = cfg['street_levels']
    
    existing = s[cfg['ch_existing']] > 0.5
    access = s[cfg['ch_access']] > 0.5
    anchors = s[cfg['ch_anchors']] > 0.5
    structure = s[cfg['ch_structure']] > 0.5
    
    # Compute legality field for visualization
    legality_loss = LocalLegalityLoss(config)
    legality_field = legality_loss.compute_legality_field(grown)[0].cpu().numpy()
    
    # Illegal structure = structure in illegal zones
    illegal = structure & (legality_field < 0.5)
    legal_struct = structure & (legality_field >= 0.5)
    
    fig = plt.figure(figsize=(20, 5))
    
    # 3D View
    ax1 = fig.add_subplot(141, projection='3d')
    if existing.any():
        ax1.voxels(existing.transpose(1,2,0), facecolors='gray', alpha=0.3)
    if anchors.any():
        anchor_disp = anchors.copy()
        anchor_disp[street_levels:,:,:] = False
        if anchor_disp.any():
            ax1.voxels(anchor_disp.transpose(1,2,0), facecolors='yellow', alpha=0.3)
    if access.any():
        ax1.voxels(access.transpose(1,2,0), facecolors='green', alpha=0.9)
    if illegal.any():
        ax1.voxels(illegal.transpose(1,2,0), facecolors='red', alpha=0.9)
    if legal_struct.any():
        ax1.voxels(legal_struct.transpose(1,2,0), facecolors='royalblue', alpha=0.6)
    ax1.set_xlabel('Y'); ax1.set_ylabel('X'); ax1.set_zlabel('Z')
    ax1.set_title(f'{title} (3D)')
    
    # Ground level
    ax2 = fig.add_subplot(142)
    plan = np.zeros((G,G,3))
    plan[existing[0,:,:]] = [0.5,0.5,0.5]
    plan[anchors[0,:,:] & ~existing[0,:,:]] = [1,1,0.5]
    plan[legal_struct.max(axis=0)] = [0.2,0.4,0.8]
    plan[illegal[0,:,:]] = [1,0,0]
    plan[access.max(axis=0)] = [0.2,0.8,0.2]
    ax2.imshow(plan.transpose(1,0,2), origin='lower')
    ax2.set_title('Ground Level')
    
    # Elevation
    ax3 = fig.add_subplot(143)
    elev = np.zeros((G,G,3))
    elev[existing.max(axis=1)] = [0.5,0.5,0.5]
    elev[legal_struct.max(axis=1)] = [0.2,0.4,0.8]
    elev[illegal.max(axis=1)] = [1,0,0]
    elev[access.max(axis=1)] = [0.2,0.8,0.2]
    ax3.imshow(elev.transpose(1,0,2), origin='lower')
    ax3.set_title('Elevation')
    
    # v2.0 Metrics
    ax4 = fig.add_subplot(144)
    ax4.axis('off')
    
    legality = compute_legality_compliance(grown, config).item()
    growth_align = compute_growth_alignment(grown, config).item()
    loadpath = compute_load_path_compliance(grown, config).item()
    access_reach = compute_access_reachability(grown, config).item()
    fill = compute_fill_ratio(grown, config).item()
    
    status_legal = 'PASS' if legality > 0.99 else 'FAIL'
    status_growth = 'PASS' if growth_align > 0.80 else 'FAIL'
    status_load = 'PASS' if loadpath > 0.95 else 'FAIL'
    status_access = 'PASS' if access_reach > 0.90 else 'FAIL'
    status_fill = 'PASS' if 0.05 < fill < 0.15 else 'FAIL'
    
    text = f"""STEP B v2.0 METRICS

Legality: {legality*100:.1f}% [{status_legal}]
(Target: 100%)

Growth Alignment: {growth_align*100:.1f}% [{status_growth}]
(Target: >80%)

Load Path: {loadpath*100:.1f}% [{status_load}]
(Target: >95%)

Access Reach: {access_reach*100:.1f}% [{status_access}]
(Target: >90%)

Fill Ratio: {fill*100:.1f}% [{status_fill}]
(Target: 5-15%)

---
Legal struct: {legal_struct.sum():.0f}
Illegal struct: {illegal.sum():.0f}
"""
    ax4.text(0.1, 0.9, text, transform=ax4.transAxes, fontsize=10,
             verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    
    return grown, meta


def plot_v2_training_curves(history):
    """Plot v2.0 training curves."""
    epochs = [h['epoch'] for h in history]
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 8))
    
    # Total loss
    axes[0,0].plot(epochs, [h['total_loss'] for h in history])
    axes[0,0].set_title('Total Loss')
    axes[0,0].set_yscale('log')
    
    # Legality loss
    axes[0,1].plot(epochs, [h['L_legality'] for h in history], 'r-')
    axes[0,1].set_title('Legality Loss')
    
    # Growth loss
    axes[0,2].plot(epochs, [h['L_growth'] for h in history], 'g-')
    axes[0,2].set_title('Growth Loss')
    
    # LoadPath loss
    axes[0,3].plot(epochs, [h['L_loadpath'] for h in history], 'purple')
    axes[0,3].set_title('LoadPath Loss')
    
    # Legality compliance
    axes[1,0].plot(epochs, [h['legality_compliance'] for h in history], 'r-')
    axes[1,0].axhline(1.0, color='k', linestyle='--', label='Target')
    axes[1,0].set_title('Legality Compliance')
    axes[1,0].set_ylim(0, 1.1)
    
    # Growth alignment
    axes[1,1].plot(epochs, [h['growth_alignment'] for h in history], 'g-')
    axes[1,1].axhline(0.80, color='k', linestyle='--', label='Target')
    axes[1,1].set_title('Growth Alignment')
    axes[1,1].set_ylim(0, 1.1)
    
    # LoadPath compliance
    axes[1,2].plot(epochs, [h['loadpath_compliance'] for h in history], 'purple')
    axes[1,2].axhline(0.95, color='k', linestyle='--', label='Target')
    axes[1,2].set_title('LoadPath Compliance')
    axes[1,2].set_ylim(0, 1.1)
    
    # Fill ratio
    axes[1,3].plot(epochs, [h['fill_ratio'] for h in history], 'orange')
    axes[1,3].axhline(0.15, color='r', linestyle='--', label='Max')
    axes[1,3].axhline(0.05, color='g', linestyle='--', label='Min')
    axes[1,3].set_title('Fill Ratio')
    axes[1,3].set_ylim(0, 0.25)
    
    plt.tight_layout()
    plt.show()


print('v2.0 visualization functions defined')

## 7. Training

In [ ]:
# Initialize
model = UrbanPavilionNCA(CONFIG).to(device)
trainer = OntologyTrainer(model, CONFIG, device)

print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')
print(f"Difficulty: {CONFIG['difficulty']}")
print(f"Epochs: {CONFIG['epochs']}")
print()
print('v2.0 Loss Weights:')
for name, weight in trainer.weights.items():
    print(f'  {name}: {weight}')

In [ ]:
# Before training
print('Before training:')
visualize_v2_result(model, trainer.scene_gen, CONFIG, device, 'Before Training')

In [ ]:
# Training loop
print('\n' + '='*70)
print('STEP B TRAINING v2.0: ONTOLOGY REVISION')
print('='*70)
print('NEW LOSSES:')
print('  - LocalLegality (per-voxel, not global ratio)')
print('  - AlignedGrowth (legal AND access zones)')
print('  - AccessConnectivity (access-seeded, not boundary)')
print('  - LoadPath (causal paths, not mass statistics)')
print('='*70)

for epoch in tqdm(range(CONFIG['epochs']), desc='Training'):
    metrics = trainer.train_epoch(epoch)
    
    if epoch % CONFIG['log_every'] == 0:
        tqdm.write(
            f"Epoch {epoch:4d} | Loss: {metrics['total_loss']:.2f} | "
            f"Legal: {metrics['legality_compliance']*100:.0f}% | "
            f"Fill: {metrics['fill_ratio']*100:.1f}% | "
            f"Load: {metrics['loadpath_compliance']*100:.0f}%"
        )
    
    if epoch > 0 and epoch % CONFIG['viz_every'] == 0:
        visualize_v2_result(model, trainer.scene_gen, CONFIG, device, f'Epoch {epoch}')
    
    if epoch > 0 and epoch % CONFIG['save_every'] == 0:
        trainer.save_checkpoint(f"{PROJECT_ROOT}/step_b/checkpoints/v2_epoch_{epoch}.pth")

print('='*70)
print('Training complete!')

## 8. Evaluation

In [ ]:
# Plot curves
plot_v2_training_curves(trainer.history)

In [ ]:
# Evaluate
print('Evaluating...')
eval_results = trainer.evaluate(n_samples=50)

print('\n' + '='*70)
print('STEP B v2.0 EVALUATION RESULTS')
print('='*70)

legal_pass = eval_results['avg_legality'] > 0.99
growth_pass = eval_results['avg_growth_alignment'] > 0.80
load_pass = eval_results['avg_loadpath'] > 0.95
access_pass = eval_results['avg_access_reach'] > 0.90
fill_pass = 0.05 < eval_results['avg_fill_ratio'] < 0.15

print(f"Legality:         {eval_results['avg_legality']*100:.1f}% {'PASS' if legal_pass else 'FAIL'} (target 100%)")
print(f"Growth Alignment: {eval_results['avg_growth_alignment']*100:.1f}% {'PASS' if growth_pass else 'FAIL'} (target >80%)")
print(f"Load Path:        {eval_results['avg_loadpath']*100:.1f}% {'PASS' if load_pass else 'FAIL'} (target >95%)")
print(f"Access Reach:     {eval_results['avg_access_reach']*100:.1f}% {'PASS' if access_pass else 'FAIL'} (target >90%)")
print(f"Fill Ratio:       {eval_results['avg_fill_ratio']*100:.1f}% {'PASS' if fill_pass else 'FAIL'} (target 5-15%)")
print('='*70)

all_pass = legal_pass and growth_pass and load_pass and access_pass and fill_pass
if all_pass:
    print('\nALL CRITERIA PASSED - Ready for medium difficulty')
else:
    print('\nSome criteria not met - Analyze for potential new attractors')

In [ ]:
# Final visualizations
print('\nFinal Results:')
for i in range(3):
    visualize_v2_result(model, trainer.scene_gen, CONFIG, device, f'Final {i+1}')

## 9. Save Final Checkpoint

In [ ]:
# Save final
trainer.save_checkpoint(f"{PROJECT_ROOT}/step_b/checkpoints/v2_ontology_easy.pth")

# Save history
with open(f"{PROJECT_ROOT}/step_b/logs/v2_training_history.json", 'w') as f:
    json.dump(trainer.history, f)

# Save eval
with open(f"{PROJECT_ROOT}/step_b/logs/v2_evaluation.json", 'w') as f:
    json.dump(eval_results, f, indent=2)

print('\nAll outputs saved')
print(f"Checkpoint: {PROJECT_ROOT}/step_b/checkpoints/v2_ontology_easy.pth")

## Summary

### Step B Training v2.0 - Ontology Revision

**Problem in v1.x:** Stable degenerate attractor formed because:
- Global ratios could be gamed
- Growth incentives conflicted with legality
- Boundary-seeded connectivity allowed perimeter exploit
- Mass-based structural metrics ignored load paths

**Solution:** Complete loss ontology restructure.

### Loss Changes

| Old Loss | Problem | New Loss | Fix |
|----------|---------|----------|-----|
| StreetVoidLoss | Global ratio | LocalLegalityLoss | Per-voxel field |
| AnchorBudgetLoss | Normalized | LocalLegalityLoss | Per-voxel field |
| DiceLoss | Misaligned | AlignedGrowthLoss | Legal ∩ Access |
| StreetConnectivityLoss | Boundary seeds | AccessConnectivityLoss | Access seeds |
| SupportRatioLoss | Mass statistics | LoadPathLoss | Causal paths |
| ConnectivityLoss | Ground as support | LoadPathLoss | Buildings + anchors |

### Weight Summary

| Loss | Weight | Purpose |
|------|--------|----------|
| LocalLegality | 30.0 | Per-voxel legality |
| AlignedGrowth | 25.0 | Growth in legal zones |
| LoadPath | 20.0 | Structural causality |
| AccessConnectivity | 15.0 | Functional circulation |
| Sparsity | 15.0 | Volume bounds |
| Cantilever | 5.0 | Local support |
| Density | 3.0 | Binary output |
| TV | 1.0 | Smoothness |

### Success Criteria

| Metric | Target |
|--------|--------|
| Legality | 100% |
| Growth alignment | >80% |
| Load path | >95% |
| Access reach | >90% |
| Fill ratio | 5-15% |

---

*NB02_AllConstraints_v2.0 - Ontology Revision - December 2025*